# LLM Iteration 3

**Aim**

Improve upon the model by providing the option to set random seeds for reproducibility and testing.

**Prompt**

Update the model to provide an option to input random seeds into the random number generator for reproducibility.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

In [1]:
import simpy
import numpy as np

In [2]:
class StrokeWard:
    def __init__(self, env, num_beds):
        self.env = env
        self.beds = simpy.Resource(env, capacity=num_beds)
        self.patient_count = 0
        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        ##################################

    def admit_patient(self, patient):
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            ##################################

            print(f'Patient {patient.id} ({patient.patient_type}) admitted to '
                  f'the stroke ward at day {self.env.now}')
            # Length of stay for all patients is 7 days
            yield self.env.timeout(7)
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(bed_leave_time - bed_arrival_time)
            ##################################
            print(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {self.env.now}')

class Patient:
    def __init__(self, id, patient_type):
        self.id = id
        self.patient_type = patient_type

def stroke_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(1.2)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Stroke")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def tia_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(9.3)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "TIA")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def complex_neurological_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(3.6)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Complex Neurological")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def other_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(3.2)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Other")
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

In [3]:
env = simpy.Environment()
stroke_ward = StrokeWard(env, num_beds=10)

# Set random seed for reproducibility
random_seed = 12345
rng = np.random.default_rng(random_seed)

env.process(stroke_arrival(env, stroke_ward, rng))
env.process(tia_arrival(env, stroke_ward, rng))
env.process(complex_neurological_arrival(env, stroke_ward, rng))
env.process(other_arrival(env, stroke_ward, rng))

# Set the run length to 2 months (default)
run_length_days = 60 * 2

print("Simulation started...")
env.run(until=run_length_days)
print("Simulation ended.")

Simulation started...
Patient 0 (Stroke) admitted to the stroke ward at day 0.22095908082453003
Patient 1 (Stroke) admitted to the stroke ward at day 0.8342160137789636
Patient 2 (Other) admitted to the stroke ward at day 1.3393877316922207
Patient 3 (Stroke) admitted to the stroke ward at day 2.421581121904481
Patient 4 (Stroke) admitted to the stroke ward at day 2.6607789252528145
Patient 5 (Stroke) admitted to the stroke ward at day 4.921597976468299
Patient 6 (Other) admitted to the stroke ward at day 5.993173631569812
Patient 7 (TIA) admitted to the stroke ward at day 5.998751745302316
Patient 8 (Other) admitted to the stroke ward at day 6.417707867853145
Patient 9 (Stroke) admitted to the stroke ward at day 6.795253958203036
Patient 0 (Stroke) discharged from the stroke ward at day 7.22095908082453
Patient 10 (Stroke) admitted to the stroke ward at day 7.22095908082453
Patient 1 (Stroke) discharged from the stroke ward at day 7.834216013778963
Patient 2 (Other) discharged from th

This updated Python code now includes an option to input random seeds into the random number generator for reproducibility in the simulation model of a stroke ward in a hospital using SimPy 4. The model allows setting a specific random seed value to control the randomness of the simulation results.

**Checks**

In [4]:
class TestCase:
    def __init__(self, env, ward, rng):
        self.env = env
        self.ward = ward
        self.env.process(stroke_arrival(env, ward, rng))
        self.env.process(tia_arrival(env, ward, rng))
        self.env.process(complex_neurological_arrival(env, ward, rng))
        self.env.process(other_arrival(env, ward, rng))

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=RUN_LENGTH)

        if self.env.now > RUN_LENGTH:
            print('The simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=RUN_LENGTH)

        for bed_use in ward.bed_in_use_store:
            if bed_use > NUM_BEDS:
                print('The number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Checks that specified length of stay correctly implemented.
        '''
        self.env.run(until=RUN_LENGTH)

        for los in ward.los_store:
            if round(los, 10) > 7:
                print('Patient length of stay exceeded limit.')
            elif round(los, 10) < 7:
                print('Patient did not stay for the full duration.')

In [5]:
# Checks run length
NUM_BEDS = 10
RUN_LENGTH = 10

random_seed = 12345
rng = np.random.default_rng(random_seed)

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rng)
test.test_run_length()

Patient 0 (Stroke) admitted to the stroke ward at day 0.22095908082453003
Patient 1 (Stroke) admitted to the stroke ward at day 0.8342160137789636
Patient 2 (Other) admitted to the stroke ward at day 1.3393877316922207
Patient 3 (Stroke) admitted to the stroke ward at day 2.421581121904481
Patient 4 (Stroke) admitted to the stroke ward at day 2.6607789252528145
Patient 5 (Stroke) admitted to the stroke ward at day 4.921597976468299
Patient 6 (Other) admitted to the stroke ward at day 5.993173631569812
Patient 7 (TIA) admitted to the stroke ward at day 5.998751745302316
Patient 8 (Other) admitted to the stroke ward at day 6.417707867853145
Patient 9 (Stroke) admitted to the stroke ward at day 6.795253958203036
Patient 0 (Stroke) discharged from the stroke ward at day 7.22095908082453
Patient 10 (Stroke) admitted to the stroke ward at day 7.22095908082453
Patient 1 (Stroke) discharged from the stroke ward at day 7.834216013778963
Patient 2 (Other) discharged from the stroke ward at day 8

In [6]:
# Checks that is only ever 1 patient if there is only 1 bed
NUM_BEDS = 1
RUN_LENGTH = 60

random_seed = 12345
rng = np.random.default_rng(random_seed)

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rng)
test.test_bed_hold()

Patient 0 (Stroke) admitted to the stroke ward at day 0.22095908082453003
Patient 0 (Stroke) discharged from the stroke ward at day 7.22095908082453
Patient 1 (Stroke) admitted to the stroke ward at day 7.22095908082453
Patient 1 (Stroke) discharged from the stroke ward at day 14.22095908082453
Patient 2 (Other) admitted to the stroke ward at day 14.22095908082453
Patient 2 (Other) discharged from the stroke ward at day 21.220959080824528
Patient 3 (Stroke) admitted to the stroke ward at day 21.220959080824528
Patient 3 (Stroke) discharged from the stroke ward at day 28.220959080824528
Patient 4 (Stroke) admitted to the stroke ward at day 28.220959080824528
Patient 4 (Stroke) discharged from the stroke ward at day 35.22095908082453
Patient 5 (Stroke) admitted to the stroke ward at day 35.22095908082453
Patient 5 (Stroke) discharged from the stroke ward at day 42.22095908082453
Patient 6 (Other) admitted to the stroke ward at day 42.22095908082453
Patient 6 (Other) discharged from the s

In [7]:
# Checks if stay length correctly implemented
NUM_BEDS = 10
RUN_LENGTH = 10

random_seed = 12345
rng = np.random.default_rng(random_seed)

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rng)
test.test_length_of_stay()

Patient 0 (Stroke) admitted to the stroke ward at day 0.22095908082453003
Patient 1 (Stroke) admitted to the stroke ward at day 0.8342160137789636
Patient 2 (Other) admitted to the stroke ward at day 1.3393877316922207
Patient 3 (Stroke) admitted to the stroke ward at day 2.421581121904481
Patient 4 (Stroke) admitted to the stroke ward at day 2.6607789252528145
Patient 5 (Stroke) admitted to the stroke ward at day 4.921597976468299
Patient 6 (Other) admitted to the stroke ward at day 5.993173631569812
Patient 7 (TIA) admitted to the stroke ward at day 5.998751745302316
Patient 8 (Other) admitted to the stroke ward at day 6.417707867853145
Patient 9 (Stroke) admitted to the stroke ward at day 6.795253958203036
Patient 0 (Stroke) discharged from the stroke ward at day 7.22095908082453
Patient 10 (Stroke) admitted to the stroke ward at day 7.22095908082453
Patient 1 (Stroke) discharged from the stroke ward at day 7.834216013778963
Patient 2 (Other) discharged from the stroke ward at day 8

In [8]:
# Testing a new random seed
env = simpy.Environment()
stroke_ward1 = StrokeWard(env, num_beds=10)

random_seed = 42
rng = np.random.default_rng(random_seed)

env.process(stroke_arrival(env, stroke_ward1, rng))
env.process(tia_arrival(env, stroke_ward1, rng))
env.process(complex_neurological_arrival(env, stroke_ward1, rng))
env.process(other_arrival(env, stroke_ward1, rng))

run_length_days = 10
env.run(until=run_length_days)
print('\n')

env = simpy.Environment()
stroke_ward2 = StrokeWard(env, num_beds=10)

random_seed = 42
rng = np.random.default_rng(random_seed)

env.process(stroke_arrival(env, stroke_ward2, rng))
env.process(tia_arrival(env, stroke_ward2, rng))
env.process(complex_neurological_arrival(env, stroke_ward2, rng))
env.process(other_arrival(env, stroke_ward2, rng))

run_length_days = 10
env.run(until=run_length_days)

for los1, los2 in zip(stroke_ward1.los_store, stroke_ward2.los_store):
    if los1 != los2:
        print('Same seed has given a different output.')

Patient 0 (Other) admitted to the stroke ward at day 0.8953417275250664
Patient 1 (Other) admitted to the stroke ward at day 1.1719414065598728
Patient 2 (Stroke) admitted to the stroke ward at day 2.8850503247591934
Patient 3 (Stroke) admitted to the stroke ward at day 4.5770031578682815
Patient 4 (Other) admitted to the stroke ward at day 5.820455056819556
Patient 5 (Other) admitted to the stroke ward at day 6.074196487806785
Patient 0 (Other) discharged from the stroke ward at day 7.895341727525066
Patient 1 (Other) discharged from the stroke ward at day 8.171941406559872
Patient 6 (Stroke) admitted to the stroke ward at day 8.326158305727677
Patient 7 (Stroke) admitted to the stroke ward at day 8.410681873885046
Patient 8 (Complex Neurological) admitted to the stroke ward at day 8.585139599547318
Patient 9 (Other) admitted to the stroke ward at day 9.423191196744911
Patient 10 (Stroke) admitted to the stroke ward at day 9.717510227513799
Patient 2 (Stroke) discharged from the strok

**Reflection**

The LLM has implemented a single random number generator using one random seed, meaning the results are the same each time the code is run as instructed. Using the same random seed results in the same outout as shown in the test above. The tests also indicate that the model is still holding patients in the beds for their length of stay. 